In [6]:
import pystoms.synthetics as syn
import numpy as np
import pandas as pd
import scipy as sp

In [7]:
iso = syn.Isotopic_Averagine_Distribution()

In [8]:
mass = 310.1
charge = 2
sigma = 1
num_peaks = 6
x = np.arange(30900,32000)/100
iso.pdf(x,mass,charge,sigma,num_peaks)

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [9]:
syn_pep = syn.SyntheticPeptideFeature(310,2,12,1,1,)

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [13]:
norm = sp.stats.norm()